# <center> Teamfight Tactics, <br> and  Predicting the Chance <br> of Winning a Game </center>

#### <center>Samuel Kahan</center>

## <center>Introduction</center>

Teamfight Tactics (TFT) is a multiplayer auto-battler game developed by Riot Games. An 'auto-battler' is a type of strategy game where players build a team of characters and then let the game play out battles automatically between different teams. TFT combines aspects of other games like chess, poker, and League of Legends into one. The game is broken up into rounds. There are three types of rounds:

    - PVP Rounds
    - PVE Rounds
    - Carousels

During PVP and PVE rounds, each player is presented with a shop of five units which they can purchase. They can then place these units onto their battlefield. Buying three of the same unit combines them into one silver unit, and three silver units combine to make a golden unit, each stronger than the last. Units have many stats, but the most important ones are generally HP and mana. When a unit's HP hits 0, it's eliminated from the battlefiend. When a unit's mana hits 100%, it casts it's ability. Each unit has an ability which has some sort of effect, whether that be dealing damage to an enemy unit or healing your own units. Mana is gained from dealing damage and taking damage, but can also be generated by items (more on items soon). 

The difference between PVP and PVE rounds is the enemy. The 'P' in PVP stands for player, and the 'E' in PVE stands for enemy. While this may seem strange as the other players ARE your enemies, PVE is a term in video games that generally refers to playing against NPC's, or non-player characters. In PVP rounds, players go up against other players, and watch their teams fight. The loser loses some of their total HP, which starts at 100. Once you hit 0 total HP, you are eliminated. In PVE rounds, players go up against NPC's. Defeating these NPC's drops gold, items, and other loot. Items can be put on units to increase their stats, and can be combined into stronger items that have additional effects. Choosing which units to put your items on and which items to build are integral parts of the strategy of TFT.

In carousels, each player picks a unit and item from a group of unit/item combinations. The units walk around in a circle (where the word carousel comes from), and two players can pick a unit at a time. A unit is picked by simply walking into it, and players with lower total health pick first.

During each game, there are three points where the player is offered one of three 'augments' to pick from. These augments have varying effects, from buffing your unit's strength, to giving gold and items. Along with these augments, every unit has different 'traits' that can also buff your team. For example, Dark Star units gain offenstive stats whenever an ally unit is eliminated, but you must play three different Dark Star units to activate this effect. These trait effects generally get stronger the more units you play. 

Of course there is much more to the game than just these things, but this is the general idea; make the strongest board you can make at any given point, and try to minimize the total HP you lose throughout the game. In this notebook, we will explore data from TFT's seventh set, analyzing different team combinations. We will use this information to attempt to predict player placements (out of 8) when using different unit/item combinations. 

## <center>Starting With The Data</center>

In this notebook, we will use Python 3 along with multiple packages such as pandas, numpy, scikit-learn, and more. Let's import everything we need, and read in the csv file we have.

In [17]:
import pandas as pd
import numpy as np
import seaborn as sb

csv = "tft_match_history.csv"

df = pd.read_csv(csv)

df.head()

C:\Users\SKaha\AppData\Local\Temp\ipykernel_135664\468972090.py:7: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv)


,placement,match_id,augment0,augment1,augment2,Set7_Assassin,Set7_Astral,Set7_Bard,Set7_Bruiser,Set7_Cannoneer,Set7_Cavalier,Set7_Dragon,Set7_Dragonmancer,Set7_Evoker,Set7_Guardian,Set7_Guild,Set7_Jade,Set7_Legend,Set7_Mage,Set7_Mirage,Set7_Mystic,Set7_Ragewing,Set7_Revel,Set7_Scalescorn,Set7_Shapeshifter,Set7_Shimmerscale,Set7_SpellThief,Set7_Starcaller,Set7_Swiftshot,Set7_Tempest,Set7_Trainer,Set7_Warrior,Set7_Whispers,TFT7_Aatrox,TFT7_Aatrox_item0,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Anivia,TFT7_Anivia_item0,TFT7_Anivia_item1,TFT7_Anivia_item2,TFT7_AoShin,TFT7_AoShin_item0,TFT7_AoShin_item1,TFT7_AoShin_item2,TFT7_Ashe,TFT7_Ashe_item0,TFT7_Ashe_item1,TFT7_Ashe_item2,TFT7_AurelionSol,TFT7_AurelionSol_item0,TFT7_AurelionSol_item1,TFT7_AurelionSol_item2,TFT7_Bard,TFT7_Bard_item0,TFT7_Bard_item1,TFT7_Bard_item2,TFT7_Braum,TFT7_Braum_item0,TFT7_Braum_item1,TFT7_Braum_item2,TFT7_Corki,TFT7_Corki_item0,TFT7_Corki_item1,TFT7_Corki_item2,TFT7_Diana,TFT7_Diana_item0,TFT7_Diana_item1,TFT7_Diana_item2,TFT7_DragonBlue,TFT7_DragonBlue_item0,TFT7_DragonBlue_item1,TFT7_DragonBlue_item2,TFT7_DragonGold,TFT7_DragonGold_item0,TFT7_DragonGold_item1,TFT7_DragonGold_item2,TFT7_DragonGreen,TFT7_DragonGreen_item0,TFT7_DragonGreen_item1,TFT7_DragonGreen_item2,TFT7_DragonPurple,TFT7_DragonPurple_item0,TFT7_DragonPurple_item1,TFT7_DragonPurple_item2,TFT7_Elise,TFT7_Elise_item0,TFT7_Elise_item1,TFT7_Elise_item2,TFT7_Ezreal,TFT7_Ezreal_item0,TFT7_Ezreal_item1,TFT7_Ezreal_item2,TFT7_Gnar,TFT7_Gnar_item0,TFT7_Gnar_item1,TFT7_Gnar_item2,TFT7_Hecarim,TFT7_Hecarim_item0,TFT7_Hecarim_item1,TFT7_Hecarim_item2,TFT7_Heimerdinger,TFT7_Heimerdinger_item0,TFT7_Heimerdinger_item1,TFT7_Heimerdinger_item2,TFT7_Illaoi,TFT7_Illaoi_item0,TFT7_Illaoi_item1,TFT7_Illaoi_item2,TFT7_Jinx,TFT7_Jinx_item0,TFT7_Jinx_item1,TFT7_Jinx_item2,TFT7_Karma,TFT7_Karma_item0,TFT7_Karma_item1,TFT7_Karma_item2,TFT7_Kayn,TFT7_Kayn_item0,TFT7_Kayn_item1,TFT7_Kayn_item2,TFT7_LeeSin,TFT7_LeeSin_item0,TFT7_LeeSin_item1,TFT7_LeeSin_item2,TFT7_Leona,TFT7_Leona_item0,TFT7_Leona_item1,TFT7_Leona_item2,TFT7_Lillia,TFT7_Lillia_item0,TFT7_Lillia_item1,TFT7_Lillia_item2,TFT7_Lulu,TFT7_Lulu_item0,TFT7_Lulu_item1,TFT7_Lulu_item2,TFT7_Nami,TFT7_Nami_item0,TFT7_Nami_item1,TFT7_Nami_item2,TFT7_Neeko,TFT7_Neeko_item0,TFT7_Neeko_item1,TFT7_Neeko_item2,TFT7_Nidalee,TFT7_Nidalee_item0,TFT7_Nidalee_item1,TFT7_Nidalee_item2,TFT7_Nunu,TFT7_Nunu_item0,TFT7_Nunu_item1,TFT7_Nunu_item2,TFT7_Olaf,TFT7_Olaf_item0,TFT7_Olaf_item1,TFT7_Olaf_item2,TFT7_Ornn,TFT7_Ornn_item0,TFT7_Ornn_item1,TFT7_Ornn_item2,TFT7_Pyke,TFT7_Pyke_item0,TFT7_Pyke_item1,TFT7_Pyke_item2,TFT7_Qiyana,TFT7_Qiyana_item0,TFT7_Qiyana_item1,TFT7_Qiyana_item2,TFT7_Ryze,TFT7_Ryze_item0,TFT7_Ryze_item1,TFT7_Ryze_item2,TFT7_Sejuani,TFT7_Sejuani_item0,TFT7_Sejuani_item1,TFT7_Sejuani_item2,TFT7_Senna,TFT7_Senna_item0,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Sett,TFT7_Sett_item0,TFT7_Sett_item1,TFT7_Sett_item2,TFT7_Shen,TFT7_Shen_item0,TFT7_Shen_item1,TFT7_Shen_item2,TFT7_Shyvana,TFT7_Shyvana_item0,TFT7_Shyvana_item1,TFT7_Shyvana_item2,TFT7_Skarner,TFT7_Skarner_item0,TFT7_Skarner_item1,TFT7_Skarner_item2,TFT7_Sona,TFT7_Sona_item0,TFT7_Sona_item1,TFT7_Sona_item2,TFT7_Soraka,TFT7_Soraka_item0,TFT7_Soraka_item1,TFT7_Soraka_item2,TFT7_Swain,TFT7_Swain_item0,TFT7_Swain_item1,TFT7_Swain_item2,TFT7_Sylas,TFT7_Sylas_item0,TFT7_Sylas_item1,TFT7_Sylas_item2,TFT7_TahmKench,TFT7_TahmKench_item0,TFT7_TahmKench_item1,TFT7_TahmKench_item2,TFT7_Talon,TFT7_Talon_item0,TFT7_Talon_item1,TFT7_Talon_item2,TFT7_Taric,TFT7_Taric_item0,TFT7_Taric_item1,TFT7_Taric_item2,TFT7_Thresh,TFT7_Thresh_item0,TFT7_Thresh_item1,TFT7_Thresh_item2,TFT7_TrainerDragon,TFT7_TrainerDragon_item0,TFT7_TrainerDragon_item1,TFT7_TrainerDragon_item2,TFT7_Tristana,TFT7_Tristana_item0,TFT7_Tristana_item1,TFT7_Tristana_item2,TFT7_Twitch,TFT7_Twitch_item0,TFT7_Twitch_item1,TFT7_Twitch_item2,TFT7_Varus,TFT7_Varus_item0,TFT7_Varus_item1,TFT7_Varus_item2,TFT7_Vladimir,TFT7_Vladimir_item0,TFT7_Vladimir_item1,TFT7_Vl

Now we have a nice pandas dataframe to work with. In order to get an idea of the dataset we're working with, let's check out all of the column names.

In [18]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print(df.columns.tolist())

['placement', 'match_id', 'augment0', 'augment1', 'augment2', 'Set7_Assassin', 'Set7_Astral', 'Set7_Bard', 'Set7_Bruiser', 'Set7_Cannoneer', 'Set7_Cavalier', 'Set7_Dragon', 'Set7_Dragonmancer', 'Set7_Evoker', 'Set7_Guardian', 'Set7_Guild', 'Set7_Jade', 'Set7_Legend', 'Set7_Mage', 'Set7_Mirage', 'Set7_Mystic', 'Set7_Ragewing', 'Set7_Revel', 'Set7_Scalescorn', 'Set7_Shapeshifter', 'Set7_Shimmerscale', 'Set7_SpellThief', 'Set7_Starcaller', 'Set7_Swiftshot', 'Set7_Tempest', 'Set7_Trainer', 'Set7_Warrior', 'Set7_Whispers', 'TFT7_Aatrox', 'TFT7_Aatrox_item0', 'TFT7_Aatrox_item1', 'TFT7_Aatrox_item2', 'TFT7_Anivia', 'TFT7_Anivia_item0', 'TFT7_Anivia_item1', 'TFT7_Anivia_item2', 'TFT7_AoShin', 'TFT7_AoShin_item0', 'TFT7_AoShin_item1', 'TFT7_AoShin_item2', 'TFT7_Ashe', 'TFT7_Ashe_item0', 'TFT7_Ashe_item1', 'TFT7_Ashe_item2', 'TFT7_AurelionSol', 'TFT7_AurelionSol_item0', 'TFT7_AurelionSol_item1', 'TFT7_AurelionSol_item2', 'TFT7_Bard', 'TFT7_Bard_item0', 'TFT7_Bard_item1', 'TFT7_Bard_item2', 'TFT

Wow, that's a lot of columns. So what do these all represent? Columns prefixed with "Set7" are the different unit traits. These columns will show how many of each trait the participant is using. Columns prefixed with "TFT7" are the different units and their item slots. Units can hold up to three items, so each column here will represent one of their items each. As we saw previously, we have a lot of missing values, so let's replace these with 0's and "None".

In [19]:
for name in df.select_dtypes("number"):
    df[name] = df[name].fillna(0)

for name in df.select_dtypes("object"):
    df[name] = df[name].fillna("None")

df.head()

,placement,match_id,augment0,augment1,augment2,Set7_Assassin,Set7_Astral,Set7_Bard,Set7_Bruiser,Set7_Cannoneer,Set7_Cavalier,Set7_Dragon,Set7_Dragonmancer,Set7_Evoker,Set7_Guardian,Set7_Guild,Set7_Jade,Set7_Legend,Set7_Mage,Set7_Mirage,Set7_Mystic,Set7_Ragewing,Set7_Revel,Set7_Scalescorn,Set7_Shapeshifter,Set7_Shimmerscale,Set7_SpellThief,Set7_Starcaller,Set7_Swiftshot,Set7_Tempest,Set7_Trainer,Set7_Warrior,Set7_Whispers,TFT7_Aatrox,TFT7_Aatrox_item0,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Anivia,TFT7_Anivia_item0,TFT7_Anivia_item1,TFT7_Anivia_item2,TFT7_AoShin,TFT7_AoShin_item0,TFT7_AoShin_item1,TFT7_AoShin_item2,TFT7_Ashe,TFT7_Ashe_item0,TFT7_Ashe_item1,TFT7_Ashe_item2,TFT7_AurelionSol,TFT7_AurelionSol_item0,TFT7_AurelionSol_item1,TFT7_AurelionSol_item2,TFT7_Bard,TFT7_Bard_item0,TFT7_Bard_item1,TFT7_Bard_item2,TFT7_Braum,TFT7_Braum_item0,TFT7_Braum_item1,TFT7_Braum_item2,TFT7_Corki,TFT7_Corki_item0,TFT7_Corki_item1,TFT7_Corki_item2,TFT7_Diana,TFT7_Diana_item0,TFT7_Diana_item1,TFT7_Diana_item2,TFT7_DragonBlue,TFT7_DragonBlue_item0,TFT7_DragonBlue_item1,TFT7_DragonBlue_item2,TFT7_DragonGold,TFT7_DragonGold_item0,TFT7_DragonGold_item1,TFT7_DragonGold_item2,TFT7_DragonGreen,TFT7_DragonGreen_item0,TFT7_DragonGreen_item1,TFT7_DragonGreen_item2,TFT7_DragonPurple,TFT7_DragonPurple_item0,TFT7_DragonPurple_item1,TFT7_DragonPurple_item2,TFT7_Elise,TFT7_Elise_item0,TFT7_Elise_item1,TFT7_Elise_item2,TFT7_Ezreal,TFT7_Ezreal_item0,TFT7_Ezreal_item1,TFT7_Ezreal_item2,TFT7_Gnar,TFT7_Gnar_item0,TFT7_Gnar_item1,TFT7_Gnar_item2,TFT7_Hecarim,TFT7_Hecarim_item0,TFT7_Hecarim_item1,TFT7_Hecarim_item2,TFT7_Heimerdinger,TFT7_Heimerdinger_item0,TFT7_Heimerdinger_item1,TFT7_Heimerdinger_item2,TFT7_Illaoi,TFT7_Illaoi_item0,TFT7_Illaoi_item1,TFT7_Illaoi_item2,TFT7_Jinx,TFT7_Jinx_item0,TFT7_Jinx_item1,TFT7_Jinx_item2,TFT7_Karma,TFT7_Karma_item0,TFT7_Karma_item1,TFT7_Karma_item2,TFT7_Kayn,TFT7_Kayn_item0,TFT7_Kayn_item1,TFT7_Kayn_item2,TFT7_LeeSin,TFT7_LeeSin_item0,TFT7_LeeSin_item1,TFT7_LeeSin_item2,TFT7_Leona,TFT7_Leona_item0,TFT7_Leona_item1,TFT7_Leona_item2,TFT7_Lillia,TFT7_Lillia_item0,TFT7_Lillia_item1,TFT7_Lillia_item2,TFT7_Lulu,TFT7_Lulu_item0,TFT7_Lulu_item1,TFT7_Lulu_item2,TFT7_Nami,TFT7_Nami_item0,TFT7_Nami_item1,TFT7_Nami_item2,TFT7_Neeko,TFT7_Neeko_item0,TFT7_Neeko_item1,TFT7_Neeko_item2,TFT7_Nidalee,TFT7_Nidalee_item0,TFT7_Nidalee_item1,TFT7_Nidalee_item2,TFT7_Nunu,TFT7_Nunu_item0,TFT7_Nunu_item1,TFT7_Nunu_item2,TFT7_Olaf,TFT7_Olaf_item0,TFT7_Olaf_item1,TFT7_Olaf_item2,TFT7_Ornn,TFT7_Ornn_item0,TFT7_Ornn_item1,TFT7_Ornn_item2,TFT7_Pyke,TFT7_Pyke_item0,TFT7_Pyke_item1,TFT7_Pyke_item2,TFT7_Qiyana,TFT7_Qiyana_item0,TFT7_Qiyana_item1,TFT7_Qiyana_item2,TFT7_Ryze,TFT7_Ryze_item0,TFT7_Ryze_item1,TFT7_Ryze_item2,TFT7_Sejuani,TFT7_Sejuani_item0,TFT7_Sejuani_item1,TFT7_Sejuani_item2,TFT7_Senna,TFT7_Senna_item0,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Sett,TFT7_Sett_item0,TFT7_Sett_item1,TFT7_Sett_item2,TFT7_Shen,TFT7_Shen_item0,TFT7_Shen_item1,TFT7_Shen_item2,TFT7_Shyvana,TFT7_Shyvana_item0,TFT7_Shyvana_item1,TFT7_Shyvana_item2,TFT7_Skarner,TFT7_Skarner_item0,TFT7_Skarner_item1,TFT7_Skarner_item2,TFT7_Sona,TFT7_Sona_item0,TFT7_Sona_item1,TFT7_Sona_item2,TFT7_Soraka,TFT7_Soraka_item0,TFT7_Soraka_item1,TFT7_Soraka_item2,TFT7_Swain,TFT7_Swain_item0,TFT7_Swain_item1,TFT7_Swain_item2,TFT7_Sylas,TFT7_Sylas_item0,TFT7_Sylas_item1,TFT7_Sylas_item2,TFT7_TahmKench,TFT7_TahmKench_item0,TFT7_TahmKench_item1,TFT7_TahmKench_item2,TFT7_Talon,TFT7_Talon_item0,TFT7_Talon_item1,TFT7_Talon_item2,TFT7_Taric,TFT7_Taric_item0,TFT7_Taric_item1,TFT7_Taric_item2,TFT7_Thresh,TFT7_Thresh_item0,TFT7_Thresh_item1,TFT7_Thresh_item2,TFT7_TrainerDragon,TFT7_TrainerDragon_item0,TFT7_TrainerDragon_item1,TFT7_TrainerDragon_item2,TFT7_Tristana,TFT7_Tristana_item0,TFT7_Tristana_item1,TFT7_Tristana_item2,TFT7_Twitch,TFT7_Twitch_item0,TFT7_Twitch_item1,TFT7_Twitch_item2,TFT7_Varus,TFT7_Varus_item0,TFT7_Varus_item1,TFT7_Varus_item2,TFT7_Vladimir,TFT7_Vladimir_item0,TFT7_Vladimir_item1,TFT7_Vl

Perfect. This will be much easier to process. The first thing to note is that we don't really need the match_ids, so let's drop that. At the same time, we can split our data into X and Y sets, the Y set will be our target for our machine learning algorithm later on. We'll end up putting them back together, but now we have a nice defined target set. Let's add a couple columns that give us the total counts for each item, trait, and unit.

In [20]:
X = df.drop(['match_id'], axis = 1)
Y = X.pop('placement')
X.fillna('', inplace = True)
num_cols = X.select_dtypes(include = np.number).columns.tolist()
cat_cols = X.select_dtypes(include = ['object','category']).columns.tolist()
traits = [t for t in num_cols if "Set7" in t]
units = [u for u in num_cols if "TFT7" in u]
augs = ['augment0', 'augment1', 'augment2']
items = [i for i in cat_cols if i not in augs]
dfu = X.nunique().to_frame().reset_index()
dfu.columns = ['Variable','DistinctCount']
uitems = {i for col in items for i in X[col].unique().tolist()}
uaugs = {a for col in augs for a in X[col].unique().tolist()}
X['items_count'] = X[items].apply(lambda row: sum(x != 'None' for x in row), axis=1)
X['traits_count'] = X[traits].sum(axis=1)
X['units_count'] = X[units].sum(axis=1)
X.head()


,augment0,augment1,augment2,Set7_Assassin,Set7_Astral,Set7_Bard,Set7_Bruiser,Set7_Cannoneer,Set7_Cavalier,Set7_Dragon,Set7_Dragonmancer,Set7_Evoker,Set7_Guardian,Set7_Guild,Set7_Jade,Set7_Legend,Set7_Mage,Set7_Mirage,Set7_Mystic,Set7_Ragewing,Set7_Revel,Set7_Scalescorn,Set7_Shapeshifter,Set7_Shimmerscale,Set7_SpellThief,Set7_Starcaller,Set7_Swiftshot,Set7_Tempest,Set7_Trainer,Set7_Warrior,Set7_Whispers,TFT7_Aatrox,TFT7_Aatrox_item0,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Anivia,TFT7_Anivia_item0,TFT7_Anivia_item1,TFT7_Anivia_item2,TFT7_AoShin,TFT7_AoShin_item0,TFT7_AoShin_item1,TFT7_AoShin_item2,TFT7_Ashe,TFT7_Ashe_item0,TFT7_Ashe_item1,TFT7_Ashe_item2,TFT7_AurelionSol,TFT7_AurelionSol_item0,TFT7_AurelionSol_item1,TFT7_AurelionSol_item2,TFT7_Bard,TFT7_Bard_item0,TFT7_Bard_item1,TFT7_Bard_item2,TFT7_Braum,TFT7_Braum_item0,TFT7_Braum_item1,TFT7_Braum_item2,TFT7_Corki,TFT7_Corki_item0,TFT7_Corki_item1,TFT7_Corki_item2,TFT7_Diana,TFT7_Diana_item0,TFT7_Diana_item1,TFT7_Diana_item2,TFT7_DragonBlue,TFT7_DragonBlue_item0,TFT7_DragonBlue_item1,TFT7_DragonBlue_item2,TFT7_DragonGold,TFT7_DragonGold_item0,TFT7_DragonGold_item1,TFT7_DragonGold_item2,TFT7_DragonGreen,TFT7_DragonGreen_item0,TFT7_DragonGreen_item1,TFT7_DragonGreen_item2,TFT7_DragonPurple,TFT7_DragonPurple_item0,TFT7_DragonPurple_item1,TFT7_DragonPurple_item2,TFT7_Elise,TFT7_Elise_item0,TFT7_Elise_item1,TFT7_Elise_item2,TFT7_Ezreal,TFT7_Ezreal_item0,TFT7_Ezreal_item1,TFT7_Ezreal_item2,TFT7_Gnar,TFT7_Gnar_item0,TFT7_Gnar_item1,TFT7_Gnar_item2,TFT7_Hecarim,TFT7_Hecarim_item0,TFT7_Hecarim_item1,TFT7_Hecarim_item2,TFT7_Heimerdinger,TFT7_Heimerdinger_item0,TFT7_Heimerdinger_item1,TFT7_Heimerdinger_item2,TFT7_Illaoi,TFT7_Illaoi_item0,TFT7_Illaoi_item1,TFT7_Illaoi_item2,TFT7_Jinx,TFT7_Jinx_item0,TFT7_Jinx_item1,TFT7_Jinx_item2,TFT7_Karma,TFT7_Karma_item0,TFT7_Karma_item1,TFT7_Karma_item2,TFT7_Kayn,TFT7_Kayn_item0,TFT7_Kayn_item1,TFT7_Kayn_item2,TFT7_LeeSin,TFT7_LeeSin_item0,TFT7_LeeSin_item1,TFT7_LeeSin_item2,TFT7_Leona,TFT7_Leona_item0,TFT7_Leona_item1,TFT7_Leona_item2,TFT7_Lillia,TFT7_Lillia_item0,TFT7_Lillia_item1,TFT7_Lillia_item2,TFT7_Lulu,TFT7_Lulu_item0,TFT7_Lulu_item1,TFT7_Lulu_item2,TFT7_Nami,TFT7_Nami_item0,TFT7_Nami_item1,TFT7_Nami_item2,TFT7_Neeko,TFT7_Neeko_item0,TFT7_Neeko_item1,TFT7_Neeko_item2,TFT7_Nidalee,TFT7_Nidalee_item0,TFT7_Nidalee_item1,TFT7_Nidalee_item2,TFT7_Nunu,TFT7_Nunu_item0,TFT7_Nunu_item1,TFT7_Nunu_item2,TFT7_Olaf,TFT7_Olaf_item0,TFT7_Olaf_item1,TFT7_Olaf_item2,TFT7_Ornn,TFT7_Ornn_item0,TFT7_Ornn_item1,TFT7_Ornn_item2,TFT7_Pyke,TFT7_Pyke_item0,TFT7_Pyke_item1,TFT7_Pyke_item2,TFT7_Qiyana,TFT7_Qiyana_item0,TFT7_Qiyana_item1,TFT7_Qiyana_item2,TFT7_Ryze,TFT7_Ryze_item0,TFT7_Ryze_item1,TFT7_Ryze_item2,TFT7_Sejuani,TFT7_Sejuani_item0,TFT7_Sejuani_item1,TFT7_Sejuani_item2,TFT7_Senna,TFT7_Senna_item0,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Sett,TFT7_Sett_item0,TFT7_Sett_item1,TFT7_Sett_item2,TFT7_Shen,TFT7_Shen_item0,TFT7_Shen_item1,TFT7_Shen_item2,TFT7_Shyvana,TFT7_Shyvana_item0,TFT7_Shyvana_item1,TFT7_Shyvana_item2,TFT7_Skarner,TFT7_Skarner_item0,TFT7_Skarner_item1,TFT7_Skarner_item2,TFT7_Sona,TFT7_Sona_item0,TFT7_Sona_item1,TFT7_Sona_item2,TFT7_Soraka,TFT7_Soraka_item0,TFT7_Soraka_item1,TFT7_Soraka_item2,TFT7_Swain,TFT7_Swain_item0,TFT7_Swain_item1,TFT7_Swain_item2,TFT7_Sylas,TFT7_Sylas_item0,TFT7_Sylas_item1,TFT7_Sylas_item2,TFT7_TahmKench,TFT7_TahmKench_item0,TFT7_TahmKench_item1,TFT7_TahmKench_item2,TFT7_Talon,TFT7_Talon_item0,TFT7_Talon_item1,TFT7_Talon_item2,TFT7_Taric,TFT7_Taric_item0,TFT7_Taric_item1,TFT7_Taric_item2,TFT7_Thresh,TFT7_Thresh_item0,TFT7_Thresh_item1,TFT7_Thresh_item2,TFT7_TrainerDragon,TFT7_TrainerDragon_item0,TFT7_TrainerDragon_item1,TFT7_TrainerDragon_item2,TFT7_Tristana,TFT7_Tristana_item0,TFT7_Tristana_item1,TFT7_Tristana_item2,TFT7_Twitch,TFT7_Twitch_item0,TFT7_Twitch_item1,TFT7_Twitch_item2,TFT7_Varus,TFT7_Varus_item0,TFT7_Varus_item1,TFT7_Varus_item2,TFT7_Vladimir,TFT7_Vladimir_item0,TFT7_Vladimir_item1,TFT7_Vladimir_item2,TFT7_V

Now that we have the item, trait, and unit counts for each team, let's recalculate the numerical and cetegorical columns we have and put the placements back on.

In [21]:
num_cols = X.select_dtypes(include=np.number).columns.tolist()
can_cols = X.select_dtypes(include=['object','category']).columns.tolist()
X[num_cols] = X[num_cols].astype(np.int64)
mdf= X.copy()
mdf['placement'] = Y
mdf.head()

,augment0,augment1,augment2,Set7_Assassin,Set7_Astral,Set7_Bard,Set7_Bruiser,Set7_Cannoneer,Set7_Cavalier,Set7_Dragon,Set7_Dragonmancer,Set7_Evoker,Set7_Guardian,Set7_Guild,Set7_Jade,Set7_Legend,Set7_Mage,Set7_Mirage,Set7_Mystic,Set7_Ragewing,Set7_Revel,Set7_Scalescorn,Set7_Shapeshifter,Set7_Shimmerscale,Set7_SpellThief,Set7_Starcaller,Set7_Swiftshot,Set7_Tempest,Set7_Trainer,Set7_Warrior,Set7_Whispers,TFT7_Aatrox,TFT7_Aatrox_item0,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Anivia,TFT7_Anivia_item0,TFT7_Anivia_item1,TFT7_Anivia_item2,TFT7_AoShin,TFT7_AoShin_item0,TFT7_AoShin_item1,TFT7_AoShin_item2,TFT7_Ashe,TFT7_Ashe_item0,TFT7_Ashe_item1,TFT7_Ashe_item2,TFT7_AurelionSol,TFT7_AurelionSol_item0,TFT7_AurelionSol_item1,TFT7_AurelionSol_item2,TFT7_Bard,TFT7_Bard_item0,TFT7_Bard_item1,TFT7_Bard_item2,TFT7_Braum,TFT7_Braum_item0,TFT7_Braum_item1,TFT7_Braum_item2,TFT7_Corki,TFT7_Corki_item0,TFT7_Corki_item1,TFT7_Corki_item2,TFT7_Diana,TFT7_Diana_item0,TFT7_Diana_item1,TFT7_Diana_item2,TFT7_DragonBlue,TFT7_DragonBlue_item0,TFT7_DragonBlue_item1,TFT7_DragonBlue_item2,TFT7_DragonGold,TFT7_DragonGold_item0,TFT7_DragonGold_item1,TFT7_DragonGold_item2,TFT7_DragonGreen,TFT7_DragonGreen_item0,TFT7_DragonGreen_item1,TFT7_DragonGreen_item2,TFT7_DragonPurple,TFT7_DragonPurple_item0,TFT7_DragonPurple_item1,TFT7_DragonPurple_item2,TFT7_Elise,TFT7_Elise_item0,TFT7_Elise_item1,TFT7_Elise_item2,TFT7_Ezreal,TFT7_Ezreal_item0,TFT7_Ezreal_item1,TFT7_Ezreal_item2,TFT7_Gnar,TFT7_Gnar_item0,TFT7_Gnar_item1,TFT7_Gnar_item2,TFT7_Hecarim,TFT7_Hecarim_item0,TFT7_Hecarim_item1,TFT7_Hecarim_item2,TFT7_Heimerdinger,TFT7_Heimerdinger_item0,TFT7_Heimerdinger_item1,TFT7_Heimerdinger_item2,TFT7_Illaoi,TFT7_Illaoi_item0,TFT7_Illaoi_item1,TFT7_Illaoi_item2,TFT7_Jinx,TFT7_Jinx_item0,TFT7_Jinx_item1,TFT7_Jinx_item2,TFT7_Karma,TFT7_Karma_item0,TFT7_Karma_item1,TFT7_Karma_item2,TFT7_Kayn,TFT7_Kayn_item0,TFT7_Kayn_item1,TFT7_Kayn_item2,TFT7_LeeSin,TFT7_LeeSin_item0,TFT7_LeeSin_item1,TFT7_LeeSin_item2,TFT7_Leona,TFT7_Leona_item0,TFT7_Leona_item1,TFT7_Leona_item2,TFT7_Lillia,TFT7_Lillia_item0,TFT7_Lillia_item1,TFT7_Lillia_item2,TFT7_Lulu,TFT7_Lulu_item0,TFT7_Lulu_item1,TFT7_Lulu_item2,TFT7_Nami,TFT7_Nami_item0,TFT7_Nami_item1,TFT7_Nami_item2,TFT7_Neeko,TFT7_Neeko_item0,TFT7_Neeko_item1,TFT7_Neeko_item2,TFT7_Nidalee,TFT7_Nidalee_item0,TFT7_Nidalee_item1,TFT7_Nidalee_item2,TFT7_Nunu,TFT7_Nunu_item0,TFT7_Nunu_item1,TFT7_Nunu_item2,TFT7_Olaf,TFT7_Olaf_item0,TFT7_Olaf_item1,TFT7_Olaf_item2,TFT7_Ornn,TFT7_Ornn_item0,TFT7_Ornn_item1,TFT7_Ornn_item2,TFT7_Pyke,TFT7_Pyke_item0,TFT7_Pyke_item1,TFT7_Pyke_item2,TFT7_Qiyana,TFT7_Qiyana_item0,TFT7_Qiyana_item1,TFT7_Qiyana_item2,TFT7_Ryze,TFT7_Ryze_item0,TFT7_Ryze_item1,TFT7_Ryze_item2,TFT7_Sejuani,TFT7_Sejuani_item0,TFT7_Sejuani_item1,TFT7_Sejuani_item2,TFT7_Senna,TFT7_Senna_item0,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Sett,TFT7_Sett_item0,TFT7_Sett_item1,TFT7_Sett_item2,TFT7_Shen,TFT7_Shen_item0,TFT7_Shen_item1,TFT7_Shen_item2,TFT7_Shyvana,TFT7_Shyvana_item0,TFT7_Shyvana_item1,TFT7_Shyvana_item2,TFT7_Skarner,TFT7_Skarner_item0,TFT7_Skarner_item1,TFT7_Skarner_item2,TFT7_Sona,TFT7_Sona_item0,TFT7_Sona_item1,TFT7_Sona_item2,TFT7_Soraka,TFT7_Soraka_item0,TFT7_Soraka_item1,TFT7_Soraka_item2,TFT7_Swain,TFT7_Swain_item0,TFT7_Swain_item1,TFT7_Swain_item2,TFT7_Sylas,TFT7_Sylas_item0,TFT7_Sylas_item1,TFT7_Sylas_item2,TFT7_TahmKench,TFT7_TahmKench_item0,TFT7_TahmKench_item1,TFT7_TahmKench_item2,TFT7_Talon,TFT7_Talon_item0,TFT7_Talon_item1,TFT7_Talon_item2,TFT7_Taric,TFT7_Taric_item0,TFT7_Taric_item1,TFT7_Taric_item2,TFT7_Thresh,TFT7_Thresh_item0,TFT7_Thresh_item1,TFT7_Thresh_item2,TFT7_TrainerDragon,TFT7_TrainerDragon_item0,TFT7_TrainerDragon_item1,TFT7_TrainerDragon_item2,TFT7_Tristana,TFT7_Tristana_item0,TFT7_Tristana_item1,TFT7_Tristana_item2,TFT7_Twitch,TFT7_Twitch_item0,TFT7_Twitch_item1,TFT7_Twitch_item2,TFT7_Varus,TFT7_Varus_item0,TFT7_Varus_item1,TFT7_Varus_item2,TFT7_Vladimir,TFT7_Vladimir_item0,TFT7_Vladimir_item1,TFT7_Vladimir_item2,TFT7_V

Now we have a dataframe where each row shows a specific team for a match. augment0, 1 and 2 refer to the augments they picked that game, columns prefixed with 'Set7' refer to the amount of that trait a team had, columns prefixed with "TFT7" and without the word 'item' refer to the amount of that champion played by a team, the same columns followed by 'item0', 1 and 2 refer to the items put on that unit, placement is the final placement of the team (out of 8), and of course the last three columns are the ones we just made. These show the total amount of traits, units, and items each team had.